In [ ]:
import pyodbc
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox



win = tk.Tk()
win.title('case study by Tạ Văn Vỹ')
win.geometry('1300x600')

frame1 = tk.Frame(win)
frame1.pack(padx=10,pady=10)

style = ttk.Style()

style.theme_use('classic')

style.configure("Treeview",
                bg = "silver",
                fg = 'black',
                rh = 25,
                fieldbackground = "white")

# Chức năng 
def change_theme():
    style.theme_use(selected_theme.get())
def theme():
    global selected_theme
    theme = tk.Toplevel(win)
    theme.title("Thay đổi Theme")
    theme.resizable(False, False)
    theme.geometry("200x250")
    style = ttk.Style()
    
    selected_theme = tk.StringVar()
    theme_frame = ttk.LabelFrame(theme, text='Themes')
    theme_frame.grid(padx=10, pady=10, ipadx=20, ipady=20, sticky='w')

    for theme_name in style.theme_names():
        rb = ttk.Radiobutton(
            theme_frame,
            text=theme_name,
            value=theme_name,
            variable=selected_theme,
            command=change_theme)
        rb.pack(expand=True, fill='both')
        
# Hàm Thoát chương trình
def exit():
    yes_no = messagebox.askyesno("Thoát CT!!!!", "Bạn muốn thoát khỏi ứng dụng ?")
    if yes_no == True:
        win.destroy()
    else:
        return
#
v_menu = tk.Menu(win)
win.config(menu=v_menu)
file_menu = tk.Menu(v_menu, tearoff=False)
v_menu.add_cascade(label="File", menu=file_menu)
file_menu.add_command(label="Sửa đổi giao diện ứng dụng", command=theme)
file_menu.add_command(label="Thoát chương trình", command=exit)


#Làm mới Ô Entry
def dele():
    sach_entry.delete(0, tk.END)
    ten_sach_entry.delete(0, tk.END)
    the_loai_entry.set('')
    nam_xuat_ban_entry.delete(0, tk.END)
    so_trang_entry.delete(0, tk.END)
    nha_xuat_ban_entry.set('')
    gia_tien_entry.delete(0, tk.END)
    
#Làm mới Bảng TreeView
def reset():
    tree.delete(*tree.get_children())
    conn = pyodbc.connect('Driver={SQL Server};'
                'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
                'Database=QuanLyThuVien;'
                'Trusted_Connection=yes;')
    cursor = conn.cursor()
    cursor.execute("""SELECT * FROM Sach""")
    df = cursor.fetchall()
    for i in df:
        tree.insert(parent='', index='end', values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]))
    dele()

# Thêm Thông tin sách
def ADD():
    conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
                      'Database=QuanLyThuVien;'
                      'Trusted_Connection=yes;')
    cursor = conn.cursor()
    cursor.execute("""INSERT INTO Sach VALUES(?,?,?,?,?,?)""",
        ten_sach_entry.get(),
        the_loai_entry.get(),
        nam_xuat_ban_entry.get(),
        so_trang_entry.get(),
        nha_xuat_ban_entry.get(),
        gia_tien_entry.get(),
    )
    
    cursor = conn.cursor()
    conn.commit()
    conn.close()

    tree.delete(*tree.get_children())
    dele()
    reset()
    messagebox.showinfo("Add!", "Bạn đã thêm thành công dữ liệu!")
    
# Update Thông tin trong bảng
def update():
    global sach_entry,ten_sach_entry,the_loai_entry,nam_xuat_ban_entry,so_trang_entry,nha_xuat_ban_entry,gia_tien_entry
    selected = tree.focus()
    tree.item(selected, text="", values=(sach_entry.get(),ten_sach_entry.get(),the_loai_entry.get(),
                                         nam_xuat_ban_entry.get(),so_trang_entry.get(),
                                         nha_xuat_ban_entry.get(),gia_tien_entry.get()))
    conn = pyodbc.connect('Driver={SQL Server};'
        'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
        'Database=QuanLyThuVien;'
        'Trusted_Connection=yes;')
    cursor = conn.cursor()

    for item in tree.selection():
        item_text = tree.item(item, "values")
        sach_entry = item_text[0]
        ten_sach_entry = item_text[1]
        the_loai_entry = item_text[2]
        nam_xuat_ban_entry = item_text[3]
        so_trang_entry = item_text[4]
        nha_xuat_ban_entry = item_text[5]
        gia_tien_entry = item_text[6]
        
    cursor.execute("""UPDATE Sach SET Ten_sach = ?, The_loai = ?,Nam_xuat_ban = ?,So_trang = ?,Nha_xuat_ban = ?,Gia_tien = ?
                    Where ID_sach = ?  """, (ten_sach_entry,the_loai_entry,nam_xuat_ban_entry,so_trang_entry,nha_xuat_ban_entry,gia_tien_entry,sach_entry))
    conn.commit()
    conn.close()
#     dele()
#     reset()
    messagebox.showinfo("Update!", "Bạn đã cập nhật thành công dữ liệu!")
# Xóa thông tin trong bảng
def delete():
    yes_no = messagebox.askyesno("Delete!!!!", "Bạn muốn xóa dữ liệu đã chọn!")

    if yes_no == 1:
        x = tree.selection()

        to_delete = []

        for i in x:
            to_delete.append(tree.item(i, 'values')[0])
            tree.delete(i)    
        conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
                          'Database=QuanLyThuVien;'
                          'Trusted_Connection=yes;')
        cursor = conn.cursor() 

        cursor.executemany("DELETE FROM Sach WHERE ID_sach = ?", [(a,) for a in to_delete])
        conn.commit()
        conn.close()
        dele()
        messagebox.showinfo("Deleted!", "Bạn đã xóa thành công dữ liệu đã chọn!")
        
# Tìm kiếm thông tin trong bảng
def NXB1():
    selected_mode = str(var.get())
    global count
    if selected_mode == 'Tên sách':
        lookup_record = search_entry.get()
        for record in tree.get_children():
            tree.delete(record)

        conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
                      'Database=QuanLyThuVien;'
                      'Trusted_Connection=yes;')

        conn = conn.cursor()

        conn.execute("exec SearchSachinfo ?", (lookup_record,))
        records = conn.fetchall()

        count = 0

        for i in records:
            if count % 2==0:
                tree.insert(parent='',index='end',iid=count,text='',values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]),tags= ('evenrow',))
            else:
                tree.insert(parent='',index='end',iid=count,text='',values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]),tags= ('oddrow',))
            count += 1


        conn.commit()

        conn.close()
    elif selected_mode == 'Thể loại':
        lookup_record = search_entry.get()
        for record in tree.get_children():
            tree.delete(record)

        conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
                      'Database=QuanLyThuVien;'
                      'Trusted_Connection=yes;')

        conn = conn.cursor()

        conn.execute("exec TheLoai ?", (lookup_record,))
        records = conn.fetchall()

        count = 0

        for i in records:
            if count % 2==0:
                tree.insert(parent='',index='end',iid=count,text='',values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]),tags= ('evenrow',))
            else:
                tree.insert(parent='',index='end',iid=count,text='',values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]),tags= ('oddrow',))
            count += 1


        conn.commit()

        conn.close()
    elif selected_mode == 'Năm xuất bản':
        lookup_record = search_entry.get()
        for record in tree.get_children():
            tree.delete(record)

        conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
                      'Database=QuanLyThuVien;'
                      'Trusted_Connection=yes;')

        conn = conn.cursor()

        conn.execute("exec NamXuatBan ?", (lookup_record,))
        records = conn.fetchall()

        count = 0

        for i in records:
            if count % 2==0:
                tree.insert(parent='',index='end',iid=count,text='',values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]),tags= ('evenrow',))
            else:
                tree.insert(parent='',index='end',iid=count,text='',values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]),tags= ('oddrow',))
            count += 1


        conn.commit()

        conn.close()
    elif selected_mode == 'Số trang':
        lookup_record = search_entry.get()
        for record in tree.get_children():
            tree.delete(record)

        conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
                      'Database=QuanLyThuVien;'
                      'Trusted_Connection=yes;')

        conn = conn.cursor()

        conn.execute("exec SoTrang ?", (lookup_record,))
        records = conn.fetchall()

        count = 0

        for i in records:
            if count % 2==0:
                tree.insert(parent='',index='end',iid=count,text='',values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]),tags= ('evenrow',))
            else:
                tree.insert(parent='',index='end',iid=count,text='',values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]),tags= ('oddrow',))
            count += 1


        conn.commit()

        conn.close()
    elif selected_mode == 'Nhà Xuất Bản':
        lookup_record = search_entry.get()
        for record in tree.get_children():
            tree.delete(record)

        conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
                      'Database=QuanLyThuVien;'
                      'Trusted_Connection=yes;')

        conn = conn.cursor()

        conn.execute("exec NXB1 ?", (lookup_record,))
        records = conn.fetchall()

        count = 0

        for i in records:
            if count % 2==0:
                tree.insert(parent='',index='end',iid=count,text='',values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]),tags= ('evenrow',))
            else:
                tree.insert(parent='',index='end',iid=count,text='',values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]),tags= ('oddrow',))
            count += 1


        conn.commit()

        conn.close()
    elif selected_mode == 'Giá Tiền':
        lookup_record = search_entry.get()
        for record in tree.get_children():
            tree.delete(record)

        conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
                      'Database=QuanLyThuVien;'
                      'Trusted_Connection=yes;')

        conn = conn.cursor()

        conn.execute("exec GiaTien ?", (lookup_record,))
        records = conn.fetchall()

        count = 0

        for i in records:
            if count % 2==0:
                tree.insert(parent='',index='end',iid=count,text='',values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]),tags= ('evenrow',))
            else:
                tree.insert(parent='',index='end',iid=count,text='',values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]),tags= ('oddrow',))
            count += 1


        conn.commit()

        conn.close()
    else:
        messagebox.showinfo("Tìm kiếm!", "Hãy chọn cách tìm kiếm!")

    
# Tạo 1 Box Để thực hiện tìm kiếm
var = tk.StringVar()
def lookup_records():
    global search_entry
    def reset():
        tree.delete(*tree.get_children())
        conn = conn = pyodbc.connect('Driver={SQL Server};'
                    'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
                    'Database=QuanLyThuVien;'
                    'Trusted_Connection=yes;')
        cursor = conn.cursor()
        cursor.execute("""SELECT * FROM Sach""")
        cursor = cursor.fetchall()
        for i in cursor:
            tree.insert(parent='', index='end', values=(i[0],i[1],i[2],i[3],i[4],i[5],i[6]))

    search = tk.Toplevel(win)
    search.title("Tìm Kiếm")
    search.resizable(False, False)
    search.geometry("300x300")
    
    search_frame = tk.LabelFrame(search, text="Tìm kiếm thông tin sách")
    search_frame.pack(padx=10, pady=10)

    search_entry = tk.Entry(search_frame, font=("Helvetica", 18))
    search_entry.grid(row = 0, column = 0,columnspan=3, padx=5, pady=5)

    search_button = tk.Button(search_frame, text="Tìm",width=10, command=NXB1)
    search_button.grid(row = 1, column = 0, padx=5, pady=5)
    
    reset = tk.Button(search_frame, text="Làm Mới",width=10, command=reset)
    reset.grid(row = 1, column = 1, padx=5, pady=5)
    
    exit = tk.Button(search_frame, text="Thoát",width=10, command=search.destroy)
    exit.grid(row = 1, column = 2, padx=5, pady=5)

    search_frame1 = tk.LabelFrame(search, text="Chọn Cách tìm thông tin")
    search_frame1.pack(padx=20, pady=20)
    
    radiobutton1 = tk.Radiobutton(search_frame1, text="Tên sách",variable=var, value='Tên sách', command=search_button)
    radiobutton1.grid(row = 0, column = 0,sticky=tk.W)

    radiobutton2 = tk.Radiobutton(search_frame1, text="Thể loại",variable=var, value='Thể loại', command=search_button)
    radiobutton2.grid(row = 0, column = 1,sticky=tk.W)
    
    radiobutton3 = tk.Radiobutton(search_frame1, text="Năm xuất bản",variable=var, value='Năm xuất bản', command=search_button)
    radiobutton3.grid(row = 1, column = 0,sticky=tk.W)

    radiobutton4 = tk.Radiobutton(search_frame1, text="Số trang",variable=var, value='Số trang', command=search_button)
    radiobutton4.grid(row = 1, column = 1,sticky=tk.W)

    radiobutton5 = tk.Radiobutton(search_frame1, text="Nhà Xuất Bản",variable=var, value='Nhà Xuất Bản', command=search_button)
    radiobutton5.grid(row = 2, column = 0,sticky=tk.W)

    radiobutton6 = tk.Radiobutton(search_frame1, text="Giá Tiền",variable=var, value='Giá Tiền', command=search_button)
    radiobutton6.grid(row = 2, column = 1,sticky=tk.W)



# Thực hiện thông tin vào ô Entry khi nhấp đúp chuột
def click(e):
    dele()
    clicka = tree.focus()
    value = tree.item(clicka, 'values')
    if value:
        sach_entry.insert(0, value[0])
        ten_sach_entry.insert(0, value[1])
        the_loai_entry.set(value[2])
        nam_xuat_ban_entry.insert(0, value[3])
        so_trang_entry.insert(0, value[4])
        nha_xuat_ban_entry.set(value[5])
        gia_tien_entry.insert(0, value[6])

# Hiện Danh sách Thể loại đã có ở SQL vào ô Entry để chọn
def tl():
    global danh_sach_tl1
    conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
                      'Database=QuanLyThuVien;'
                      'Trusted_Connection=yes;')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM Loai_sach')
    danh_sach_tl = cursor.fetchall()
    danh_sach_tl1 = []
    for i in danh_sach_tl:
        danh_sach_tl1 += i[1:6]

# Hiện Danh sách Nhà xuất bản đã có ở SQL vào ô Entry để chọn
def nxb():
    global danh_sach_nxb
    conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-FCU7LQB\SQLEXPRESS;'
                      'Database=QuanLyThuVien;'
                      'Trusted_Connection=yes;')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM NXB')
    danh_sach = cursor.fetchall()
    danh_sach_nxb = []
    for i in danh_sach:
        danh_sach_nxb += i[1:6]

#Tên sách
ten_sach = tk.Label(frame1,text = 'Tên sách:', font=("Helvetica", 12))
ten_sach.grid(row = 0, column = 0,sticky=tk.E, padx=5, pady=5)
ten_sach_entry = tk.Entry(frame1)
ten_sach_entry.grid(row = 0, column = 1, ipadx=7, ipady=2)


#Thể Loại
tl()
the_loai = tk.Label(frame1,text = 'Thể Loại:', font=("Helvetica", 12))
the_loai.grid(row = 1, column = 0,sticky=tk.E, padx=5, pady=5)
the_loai_entry = ttk.Combobox(frame1)
the_loai_entry['values'] = danh_sach_tl1
the_loai_entry['state'] = 'readonly'
the_loai_entry.grid(row = 1, column = 1, padx=5, pady=5)

#Năm Xuất Bản
nam_xuat_ban = tk.Label(frame1,text = 'Năm Xuất Bản:', font=("Helvetica", 12))
nam_xuat_ban.grid(row = 0, column = 2,sticky=tk.E, padx=5, pady=5)
nam_xuat_ban_entry = tk.Entry(frame1)
nam_xuat_ban_entry.grid(row = 0, column = 3, ipadx=7, ipady=2)

#Số Lượng Trang
so_trang = tk.Label(frame1,text = 'Số Lượng Trang:', font=("Helvetica", 12))
so_trang.grid(row = 1, column = 2,sticky=tk.E, padx=5, pady=5)
so_trang_entry = tk.Entry(frame1)
so_trang_entry.grid(row = 1, column = 3, ipadx=7, ipady=2)


#Nhà Xuất Bản
nxb()
nha_xuat_ban = tk.Label(frame1,text = 'Nhà Xuất Bản:', font=("Helvetica", 12))
nha_xuat_ban.grid(row = 0, column = 4,sticky=tk.E, padx=5, pady=5)
nha_xuat_ban_entry = ttk.Combobox(frame1)
nha_xuat_ban_entry['values'] = danh_sach_nxb
nha_xuat_ban_entry['state'] = 'readonly'
nha_xuat_ban_entry.grid(row = 0, column = 5, ipadx=20, ipady=1)


#Giá Tiền
gia_tien = tk.Label(frame1,text = 'Giá Tiền:', font=("Helvetica", 12))
gia_tien.grid(row = 1, column = 4,sticky=tk.E, padx=5, pady=5)
gia_tien_entry = tk.Entry(frame1)
gia_tien_entry.grid(row = 1, column = 5, ipadx=30, ipady=2)

#ID Sách
sach = tk.Label(frame1,text = 'ID Sách:', font=("Helvetica", 12))
# sach.grid(row = 2, column = 0,sticky=tk.E, padx=5, pady=5)
sach_entry = tk.Entry(frame1)
# sach_entry.grid(row = 2, column = 1, padx=5, pady=5)

#Tìm kiếm
search_button = tk.Button(frame1, text = 'Tìm Kiếm', font=("Helvetica", 12), command = lookup_records,width=15, padx=5, pady=5)
search_button.grid(row = 3, column = 5,sticky=tk.W, padx=5, pady=5)

#Thêm thông tin sách vào danh sách
luu_sp = tk.Button(frame1, text='Thêm thông tin sách', font=("Helvetica", 12), width=20,command = ADD)
luu_sp.grid(row = 0, column=6, padx=5, pady=5)

#Xóa thông tin sách đã chọn
delete = tk.Button(frame1, text='Xóa thông tin sách', font=("Helvetica", 12), width=20,command = delete)
delete.grid(row = 1, column=6, padx=5, pady=5)

#Update lại thông tin sách
delete_all = tk.Button(frame1, text='Sửa thông tin sách', font=("Helvetica", 12), width=20 ,command = update)
delete_all.grid(row = 2, column=6, padx=5, pady=5)

#Làm mới bảng
xuat_sp = tk.Button(frame1, text='Làm mới', font=("Helvetica", 12), width=20, command = reset)
xuat_sp.grid(row = 3, column=6, padx=5, pady=5)



#treeview
labelframe3 = tk.LabelFrame(frame1, text='Thông tin sách', font=("Helvetica", 12))
labelframe3.grid(row = 5, column = 0,columnspan=7, sticky='nsew')
columns = ('ID sách','Tên sách','Thể loại sách','Năm xuất bản','Số trang sách','Nhà xuất bản','Giá Tiền')
tree = ttk.Treeview(labelframe3, columns=columns, show='headings')
for name in columns:
    if name == columns[2]:
        tree.column(name, width=100, anchor=tk.W)
        tree.heading(name, text=name.capitalize(), anchor=tk.CENTER)
    elif name == columns[5]:
        tree.column(name, width=200, anchor=tk.W)
        tree.heading(name, text=name.capitalize(), anchor=tk.CENTER)
    else:
        tree.column(name, width=60, anchor=tk.CENTER)
        tree.heading(name, text=name.capitalize(), anchor=tk.CENTER)
tree.pack(ipadx=190, ipady=70,pady=5, expand=True, side=tk.LEFT)

scrollbar = ttk.Scrollbar(labelframe3, orient=tk.VERTICAL)

tree.configure(yscroll=scrollbar.set)

scrollbar.pack(pady=5, fill=tk.BOTH, expand=True)

# tree.bind('<<TreeviewSelect>>', click)
tree.bind('<Double-Button-1>', click)


tree.pack()
tree.focus_set()

reset()


win.mainloop()